<a href="https://colab.research.google.com/github/LokiAndere/After_MIARFID/blob/main/image_derivative_kernels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
def lateef_sobel_weighted(s, axis = 0, dir = 1):

    k = np.zeros((s, s))
    k_lil = np.zeros((s, s))

    s=s//2
    n = 1

    for j in range(s+1):
        for i in range(1,s+1):
            d = i*i + j*j
            n = np.lcm(n, int(d / np.gcd(d, i)))

    weight = 0

    for y in range(-s,s+1):
        for x in range(-s,s+1):
            if axis:
                i, j = x, y   # "i, j = y, x" for transpose
            else:
                i, j = y, x
            d = i*i + j*j
            if (i != 0):
                if (n * i % d) != 0:
                    print('this should never happen')
                    pass
                w = n * i / d
                weight += i * w
            else:
                w = 0
            k[x+s,y+s] = w
            k_lil[x+s,y+s] = i / d if d > 0 else 0
    
    f = np.gcd(n, int(weight))

    if dir:
        k = np.fliplr(k) if axis==0 else np.flipud(k)
        k_lil = np.fliplr(k_lil) if axis==0 else np.flipud(k_lil)

    if f < n:
        f = n / f
    weight_lil = weight / f
    
    return k, weight, k_lil, weight_lil

In [ ]:
from scipy import signal

In [ ]:
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================
direct = np.asarray([1, 0, -1], dtype=np.float64)
#==================================================
blur_gauss3 = np.asarray([1, 2, 1], dtype=np.float64) # 4
#==================================================
blur_gauss03x03 = np.outer(blur_gauss3, blur_gauss3) # 4 ** 2 = 16
blur_gauss5 = signal.convolve(blur_gauss3, blur_gauss3) # 4 * 4 = 16 (16**1)
blur_gauss05x05 = np.outer(blur_gauss5, blur_gauss5) # 16 ** 2 = 256 (16**2)
blur_gauss7 = signal.convolve(blur_gauss5, blur_gauss3) # 16 * 4 = 64
blur_gauss07x07 = np.outer(blur_gauss7, blur_gauss7) # 64 ** 2 = 4096 (16**3)
blur_gauss9 = signal.convolve(blur_gauss7, blur_gauss3) # 64 * 4 = 256
blur_gauss09x09 = np.outer(blur_gauss9, blur_gauss9) # 256 ** 2 = 65536 (16**4)
blur_gauss11 = signal.convolve(blur_gauss9, blur_gauss3) # 256 * 4 = 1024
blur_gauss11x11 = np.outer(blur_gauss11, blur_gauss11) # 1024 ** 2 = 1048576 (16**5)
#==================================================
print('GAUSSIAN BLUR')
#==================================================
# print(blur_gauss3)
# print(blur_gauss3.sum())
# print(blur_gauss5)
# print(blur_gauss5.sum())
# print(blur_gauss7)
# print(blur_gauss7.sum())
# print(blur_gauss9)
# print(blur_gauss9.sum())
# print(blur_gauss11)
# print(blur_gauss11.sum())
# print(blur_gauss03x03)
# print(blur_gauss05x05)
# print(blur_gauss07x07)
# print(blur_gauss09x09)
# print(blur_gauss11x11)
#==================================================
# print(blur_gauss03x03)
# print(blur_gauss05x05)
# print(blur_gauss07x07)
#==================================================
blur_gauss = [blur_gauss03x03, blur_gauss05x05, blur_gauss07x07, blur_gauss09x09, blur_gauss11x11]
# gblur_weight = [(4 ** (i + 1)) ** 2 for i, _ in enumerate(gblur)]
blur_gauss_weight = [16 ** (i + 1) for i, _ in enumerate(blur_gauss)]
#==================================================
# print([4**2, 16**2, 64**2, 256**2, 1024**2])
# print([16**1, 16**2, 16**3, 16**4, 16**5])
# print([16, 256, 4096, 65536, 1048576])
# print(blur_gauss_weight)
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================
# blur_box03x03 = np.ones(9).reshape(3,3)
blur_box03x03 = np.ones((3,3))
#==================================================
print('BOX BLUR')
#==================================================
# print(blur_box03x03)
#==================================================
blur_box = [blur_box03x03]
blur_box_weight = [9]
#==================================================
# print(blur_box_weight)
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================
# np.sum(np.abs(direct)) = 2
# 1D finite differences kernel is 1/2 * [1 0 -1]
# and the 1D smoothing kernel is 1/4 * [ 1 2 1 ]
# the 03x03 needs a scale of 1/2 * 1/4 = 1/8
# the larger kernels need 1/4 * 1/4 = 1/16 when moving to higher dimensions
# np.sum(blur_gaus03x03) = 16
sobel_intel03x03 = np.outer(blur_gauss3, direct) # 2 * 4 = 8
sobel_intel05x05 = signal.convolve2d(sobel_intel03x03, blur_gauss03x03) # 8 * 16 = 128
sobel_intel07x07 = signal.convolve2d(sobel_intel05x05, blur_gauss03x03) # 128 * 16 = 2048
sobel_intel09x09 = signal.convolve2d(sobel_intel07x07, blur_gauss03x03) # 2048 * 16 = 32768
sobel_intel11x11 = signal.convolve2d(sobel_intel09x09, blur_gauss03x03) # 32768 * 16 = 524288
#==================================================
print('SOBEL INTEL / OPENCV')
#==================================================
# print(sobel_intel03x03)
# print(sobel_intel05x05)
# print(sobel_intel07x07)
# print(np.sum(np.abs(sobel_intel07x07))) # 1280
# print(sobel_intel09x09)
# print(sobel_intel11x11)
#==================================================
sobel_intel = [sobel_intel03x03, sobel_intel05x05, sobel_intel07x07, sobel_intel09x09, sobel_intel11x11]
sobel_intel_weight = [8 * 16 ** i for i, _ in enumerate(sobel_intel)]
#==================================================
# print([8, 128, 2048, 32768, 524288])
# print(sobel_intel_weight)
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================
heavy_weight3 = np.asarray([1, 5, 1], dtype=np.float64) # 7
heavy_weight3x3 = np.outer(heavy_weight3, heavy_weight3) # 7 * 7 = 49
#==================================================
sobel_heavy03x03 = np.outer(heavy_weight3, direct) # 2 * 7 = 14
sobel_heavy05x05 = signal.convolve2d(sobel_heavy03x03, heavy_weight3x3) # 14 * 49 = 686
sobel_heavy07x07 = signal.convolve2d(sobel_heavy05x05, heavy_weight3x3) # 686 * 49 = 33614
sobel_heavy09x09 = signal.convolve2d(sobel_heavy07x07, heavy_weight3x3) # 33614 * 49 = 1647086
sobel_heavy11x11 = signal.convolve2d(sobel_heavy09x09, heavy_weight3x3) # 1647086 * 49 = 80707214
#==================================================
print('SOBEL HEAVY WEIGHT')
#==================================================
# print(sobel_heavy03x03)
# print(sobel_heavy05x05)
# print(sobel_heavy07x07)
# print(np.sum(np.abs(sobel_heavy07x07))) # 25382
# print(sobel_heavy09x09)
# print(sobel_heavy11x11)
#==================================================
sobel_heavy = [sobel_heavy03x03, sobel_heavy05x05, sobel_heavy07x07, sobel_heavy09x09, sobel_heavy11x11]
sobel_heavy_weight = [2 * 7 ** (2 * i + 1) for i, _ in enumerate(sobel_heavy)]
#==================================================
# print([14, 686, 33614, 1647086, 80707214])
# print(sobel_heavy_weight)
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================
sobel_labview03x03 = np.asarray([
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
]) # 8
#==================================================
sobel_labview05x05 = np.asarray([
		[0, 1, 0, -1, 0],
		[1, 2, 0, -2, -1],
		[1, 2, 0, -2, -1],
		[1, 2, 0, -2, -1], 
		[0, 1, 0, -1, 0]
]) # 22
#==================================================
sobel_labview07x07 = np.asarray([
		[0, 1, 1, 0, -1, -1, 0],
		[1, 2, 2, 0, -2, -2, -1],
		[1, 2, 3, 0, -3, -2, -1],
		[1, 2, 3, 0, -3, -2, -1], 
		[1, 2, 3, 0, -3, -2, -1],
		[1, 2, 2, 0, -2, -2, -1],
		[0, 1, 1, 0, -1, -1, 0]
]) # 64
#==================================================
print('SOBEL LABVIEW')
#==================================================
# print(sobel_labview03x03)
# print(sobel_labview05x05)
# print(sobel_labview07x07)
#==================================================
sobel_labview = [sobel_labview03x03, sobel_labview05x05, sobel_labview07x07]
sobel_labview_weight = [8, 22, 64]
#==================================================
# print([8, 22, 64])
# print(sobel_labview_weight)
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================
aprox3 = np.asarray([1, 1, 1], dtype=np.float64)
aprox03x03 = np.outer(aprox3, aprox3) # 9
#==================================================
prewitt03x03 = np.outer(aprox3, direct) # 2 * 3 = 6
prewitt05x05 = signal.convolve2d(prewitt03x03, aprox03x03) # 6 * 9 = 54
prewitt07x07 = signal.convolve2d(prewitt05x05, aprox03x03) # 54 * 9 = 486
prewitt09x09 = signal.convolve2d(prewitt07x07, aprox03x03) # 486 * 9 = 4374
prewitt11x11 = signal.convolve2d(prewitt09x09, aprox03x03) # 4374 * 9 = 39366
#==================================================
print('PREWITT')
#==================================================
# print(prewitt03x03)
# print(prewitt05x05)
# print(prewitt07x07)
# print(np.sum(np.abs(prewitt07x07))) # 270
# print(prewitt09x09)
# print(prewitt11x11)
#==================================================
prewitt = [prewitt03x03, prewitt05x05, prewitt07x07, prewitt09x09, prewitt11x11]
prewitt_weight = [6 * 9 ** i for i, _ in enumerate(prewitt)]
#==================================================
# print([6, 54, 486, 4374, 39366])
# print(prewitt_weight)
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================
rotsim3 = np.asarray([3, 10, 3], dtype=np.float64) # 16
rotsim03x03 = np.outer(rotsim3, rotsim3) # 256
#==================================================
scharr03x03 = np.outer(rotsim3, direct) # 2 * 16 = 32
scharr05x05 = signal.convolve2d(scharr03x03, rotsim03x03) # 32 * 256 = 8192
scharr07x07 = signal.convolve2d(scharr05x05, rotsim03x03) # 8192 * 256 = 2097152
scharr09x09 = signal.convolve2d(scharr07x07, rotsim03x03) # 2097152 * 256 = 536870912
scharr11x11 = signal.convolve2d(scharr09x09, rotsim03x03) # 536870912 * 256 = 137438953472
#==================================================
print('SCHARR')
#==================================================
# print(scharr03x03)
# print(scharr05x05)
# print(scharr07x07)
# print(np.sum(np.abs(scharr07x07))) # 1458176
# print(scharr09x09)
# print(scharr11x11)
#==================================================
scharr = [scharr03x03, scharr05x05, scharr07x07, scharr09x09, scharr11x11]
scharr_weight = [32 * 256 ** i for i, _ in enumerate(scharr)]
#==================================================
# print([32, 8192, 2097152, 536870912, 137438953472])
# print(scharr_weight)
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================
rotsim8bit3 = np.asarray([47, 162, 47], dtype=np.float64) # 256
rotsim8bit03x03 = np.outer(rotsim8bit3, rotsim8bit3) # 65536
#==================================================
scharr8bit03x03 = np.outer(rotsim8bit3, direct) # 2 * 65536 = 131072
scharr8bit05x05 = signal.convolve2d(scharr8bit03x03, rotsim8bit03x03) # 131072 * 65536 = 8589934592 (too much)
scharr8bit07x07 = signal.convolve2d(scharr8bit05x05, rotsim8bit03x03) # too much
scharr8bit09x09 = signal.convolve2d(scharr8bit07x07, rotsim8bit03x03) # too much
scharr8bit11x11 = signal.convolve2d(scharr8bit09x09, rotsim8bit03x03) # too much
#==================================================
print('SCHARR 8 BIT')
#==================================================
# print(scharr8bit03x03)
#==================================================
scharr8bit = [scharr8bit03x03, scharr8bit05x05, scharr8bit07x07, scharr8bit09x09, scharr8bit11x11]
scharr8bit_weight = [2 * 65536 ** (i + 1) for i, _ in enumerate(scharr8bit)]
#==================================================
# print([131072])
# print(scharr8bit_weight)
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================
sobel_lateef03x03, w3, _, _ = lateef_sobel_weighted(3)
sobel_lateef05x05, w5, _, _ = lateef_sobel_weighted(5)
sobel_lateef07x07, w7, _, _ = lateef_sobel_weighted(7)
sobel_lateef09x09, w9, _, _ = lateef_sobel_weighted(9)
sobel_lateef11x11, w11, _, _ = lateef_sobel_weighted(11)
#==================================================
print('SOBEL LATEEF')
#==================================================
# print(sobel_lateef03x03)
# print(sobel_lateef03x03/4)
# print(sobel_lateef03x03*2)
# print(sobel_lateef03x03)
# print(sobel_lateef05x05)
# print(sobel_lateef07x07)
# print(sobel_lateef09x09)
# print(sobel_lateef11x11)
#==================================================
sobel_lateef = [sobel_lateef03x03, sobel_lateef05x05, sobel_lateef07x07, sobel_lateef09x09, sobel_lateef11x11]
sobel_lateef_weight = [w3, w5, w7, w9, w11]
#==================================================
# print(sobel_lateef_weight)
#==================================================
#/////////////////////////\\\\\\\\\\\\\\\\\\\\\\\\\
#\\\\\\\\\\\\\\\\\\\\\\\\\/////////////////////////
#==================================================